# Using Q\# with Python

## Abstract

This notebook shows how to use Q# and Python together to develop and simulate quantum programs with the Quantum Development Kit.

## The `qsharp` package

We begin by importing the `qsharp` Python package. This package provides Q# interoperability for Python, as well as integration with Jupyter.

In [1]:
import qsharp

Preparing Q# environment...
...

Once imported, the `qsharp` package exposes Q# functions and operations as Python objects that we can use to simulate quantum programs, estimate quantum resources, and so forth.

Let's start by defining a new Q# operation with the `%%qsharp` magic command.

In [2]:
%%qsharp

operation SayHello(name : String) : Unit {
    Message($"Hello, {name}!");
}

This creates a new Python object `SayHello` that represents our new Q# operation.

In [3]:
SayHello

<Q# callable SayHello>

By using the `.simulate` method, we can run our new operation on a quantum simulator:

In [4]:
SayHello.simulate(name="quantum world")

Hello, quantum world!


()

Inputs to your Q# operation are represented by Python keyword arguments, and outputs are returned back to the Python host.

In [5]:
%%qsharp

function Plus(x : Int, y : Int) : Int {
    return x + y;
}

In [6]:
Plus.simulate(x=3, y=5)

8

Of course, you can also use qubits in the Q# programs that you simulate from Python:

In [7]:
%%qsharp
open Microsoft.Quantum.Measurement;

operation SampleQrng() : Result {
    use q = Qubit();
    H(q);
    return MResetZ(q);
}

In [8]:
SampleQrng.simulate()

1

The `qsharp` package also provides the `compile` function, which allows for compiling Q# code from Python strings.

In [9]:
sample_qrng = qsharp.compile("""
    operation SampleQrng() : Result {
        use q = Qubit();
        H(q);
        return MResetZ(q);
    }
""")

In [10]:
sample_qrng.simulate()

0

## Workspaces, Projects, and Packages


As your quantum programs get larger, it can be inconvienent to keep all of your Q# code in a single notebook or Python script. Instead, the `qsharp` package allows you to call into Q# code from source code in the same directory as your notebook.

For example, in the cell below, `PrepareBellPair` is defined in [`./Operations.qs`](./Operations.qs), but we can call it from Q# operations defined using `%%qsharp`.

In [11]:
%%qsharp
open Microsoft.Quantum.Samples;

operation PrepareAndMeasureBellPair() : (Result, Result) {
    use left = Qubit();
    use right = Qubit();

    PrepareBellPair(left, right);
    return (MResetZ(left), MResetZ(right));
}

In [12]:
PrepareAndMeasureBellPair.simulate()

(0, 0)


After importing the `qsharp` package, you can also import Q# namespaces as though they were Python packages. For example, `Operations.qs` also defines an operation that demonstrates how to run quantum teleportation; we can import it here and run the `RunTeleportationExample` operation on our simulator.

In [13]:
from Microsoft.Quantum.Samples import RunTeleportationExample

In [14]:
RunTeleportationExample.simulate()

Teleported successfully!


()

The Q# code in your workspace can also depend on other Q# _packages_ and _projects_ by using project files.

> **TIP**: If you don't have a project file for your workspace, the `qsharp` package will assume some reasonable defaults. Having a project file makes it easy to use additional packages, to get code completion and hover documentation while you edit your Q# files, and so forth.

For example, this sample includes a Q# project file that tells the `qsharp` Python package what other Q# packages are needed to run your code.

In [15]:
with open('python.csproj') as f:
    print(f.read())

<!-- This line declares that the project is a Q# project, built using the
     Microsoft.Quantum.Sdk package. -->
<Project Sdk="Microsoft.Quantum.Sdk/0.24.210930">

  <PropertyGroup>
    <OutputType>Library</OutputType>
    <TargetFramework>net6.0</TargetFramework>
    <!-- This property tells the qsharp package to automatically load
         your project and use it as a workspace. -->
    <IQSharpLoadAutomatically>true</IQSharpLoadAutomatically>
  </PropertyGroup>

  <!-- To include functions and operations from other Q# packages, you can
       add them as package references in your project file. These packages
       will automatically be downloaded from nuget.org and made available to your
       Q# workspace. -->
  <ItemGroup>
    <PackageReference Include="Microsoft.Quantum.Numerics" Version="0.24.210930" />
  </ItemGroup>

</Project>



To see what packages are currently added to your workspace, you can use the `qsharp.packages` object:

In [16]:
qsharp.packages

Adding package .

[('Microsoft.Quantum.Standard', LooseVersion ('0.24.216242-beta')), ('Microsoft.Quantum.Standard.Visualization', LooseVersion ('0.24.216242-beta')), ('Microsoft.Quantum.Numerics', LooseVersion ('0.24.216242-beta'))]

You can also add new packages dynamically by using `qsharp.packages.add`:

In [17]:
qsharp.packages.add('Microsoft.Quantum.Chemistry')

Adding package Microsoft.Quantum.Chemistry.

In [18]:
qsharp.packages

Adding package .

[('Microsoft.Quantum.Standard', LooseVersion ('0.24.216242-beta')), ('Microsoft.Quantum.Standard.Visualization', LooseVersion ('0.24.216242-beta')), ('Microsoft.Quantum.Numerics', LooseVersion ('0.24.216242-beta')), ('Microsoft.Quantum.Chemistry', LooseVersion ('0.24.216242-beta'))]

## Diagnostics

Just as with Q# standalone notebooks, you can also use diagnostics like `DumpMachine` and `DumpOperation` from Python notebooks to learn how your Q# program work and to help diagnose issues and bugs in your Q# programs.

In [19]:
%%qsharp
open Microsoft.Quantum.Diagnostics;

operation DumpPlusState() : Unit {
    use q = Qubit();
    within {
        H(q);
    } apply {
        DumpMachine();
    }
}

In [20]:
DumpPlusState.simulate()

|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0.7071067811865476 + 0𝑖

()

> To learn more about the diagnostics features offered by Q# and the Quantum Development Kit, check out the [visualization sample](../../diagnostics/visualization/Visualizing%20Quantum%20Programs.ipynb).

The Q# package also allows you to capture these diagnostics and manipulate them as Python objects:

In [21]:
with qsharp.capture_diagnostics() as diagnostics:
    DumpPlusState.simulate()

In [22]:
diagnostics

[{'diagnostic_kind': 'state-vector',
  'qubit_ids': [0],
  'n_qubits': 1,
  'amplitudes': {'0': {'Real': 0.7071067811865476,
    'Imaginary': 0.0,
    'Magnitude': 0.7071067811865476,
    'Phase': 0.0},
   '1': {'Real': 0.7071067811865476,
    'Imaginary': 0.0,
    'Magnitude': 0.7071067811865476,
    'Phase': 0.0}}}]

Working with raw JSON for diagnostics can be somewhat inconvienent, so the `capture_diagnostics` function also supports converting diagnostics into quantum objects using the QuTiP library:

In [23]:
with qsharp.capture_diagnostics(as_qobj=True) as diagnostics:
    DumpPlusState.simulate()

In [24]:
diagnostics[0]

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.70710678]
 [0.70710678]]

## Using other simulators

Other simulators such as the Toffoli simulator, resources estimator, and open systems simulator are available as methods on the Python representations of Q# callables. For example, you can run Q# programs on the Toffoli simulator by using the `toffoli_simulate` method:

In [25]:
%%qsharp
open Microsoft.Quantum.Diagnostics;

operation DumpOneState() : Unit {
    use q = Qubit();
    within {
        X(q);
    } apply {
        DumpMachine();
    }
}

In [26]:
DumpOneState.toffoli_simulate()

Offset  	State Data
========	==========
00000000	1


()

Similarly, the resources estimator can be used with the `estimate_resources()` method:

In [27]:
PrepareAndMeasureBellPair.estimate_resources()

{'CNOT': 1,
 'QubitClifford': 1,
 'R': 0,
 'Measure': 2,
 'T': 0,
 'Depth': 0,
 'Width': 2,
 'QubitCount': 2,
 'BorrowedWidth': 0}

To use the open systems simulator, you can use the `.simulate_noise` method:

In [28]:
DumpOneState.simulate_noise()

# of qubits,3
State data,$$ \left( \begin{matrix} 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 1 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i \end{matrix} \right) $$


()

> To learn more about using Q# and Python together, please see:
> - [`qsharp` API reference](https://docs.microsoft.com/python/qsharp-core/qsharp)
>
> To learn more about using the open systems simulator, please see:
> - [Process tomography sample](../../characterization/process-tomography/README.md)
> - [Open systems simulator documentation](https://docs.microsoft.com/azure/quantum/machines/noise-simulator)